In [1]:
# Lets Import all our packages nesscary for data importing
import datetime
import os
import shutil
from decimal import Decimal

import fsspec
import pandas as pd
from nautilus_trader.core.datetime import dt_to_unix_nanos
from nautilus_trader.model.data import QuoteTick
from nautilus_trader.model.objects import Price, Quantity

from nautilus_trader.backtest.node import BacktestNode, BacktestVenueConfig, BacktestDataConfig, BacktestRunConfig, BacktestEngineConfig
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.test_kit.providers import TestInstrumentProvider


In [2]:
from nautilus_trader.persistence.catalog import ParquetDataCatalog
catalog = ParquetDataCatalog.from_env()
catalog.instruments()

[CurrencyPair(id=EUR/USD.SIM, raw_symbol=EUR/USD, asset_class=FX, asset_type=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)]

### Writing a HMA Trading Strategy

In [3]:
from typing import Optional
from nautilus_trader.core.message import Event
from nautilus_trader.indicators.average.hma import HullMovingAverage
from nautilus_trader.model.data import QuoteTick
from nautilus_trader.model.enums import PriceType
from nautilus_trader.model.enums import PositionSide
from nautilus_trader.model.enums import OrderSide
from nautilus_trader.model.events import PositionOpened
from nautilus_trader.model.identifiers import InstrumentId
from nautilus_trader.model.objects import Quantity
from nautilus_trader.model.position import Position
from nautilus_trader.trading.strategy import Strategy, StrategyConfig

In [4]:
class HMAConfig(StrategyConfig):
    instrument_id: str
    period: int = 16
    price_type: PriceType = PriceType.MID
    trade_size: int = 1_000

class HMAStrategy(Strategy):
    def __init__(self, config: HMAConfig):
        super().__init__(config=config)
        # Our "trading signal"
        self.hma = HullMovingAverage(period = config.period,
        price_type= config.price_type)
        self.instrument_id = InstrumentId.from_str(config.instrument_id)
        self.trade_size = Quantity.from_int(config.trade_size)

    # Convenience
        self.position: Optional[Position] = None
    # On start
    def on_start(self):
        # subscribe to data  
        self.subscribe_quote_ticks(instrument_id=self.instrument_id)

    # On end
    def on_stop(self):
        # Close all positions
        self.close_all_positions(self.instrument_id)
        # stop subscribing to data
        self.unsubscribe_quote_ticks(instrument_id=self.instrument_id)

    def on_quote_tick(self, tick:QuoteTick):
        # Updates indicator with quote tick 
        self.hma.handle_quote_tick(tick)
        self.bid = tick.bid_price
        if not self.hma.initialized:
            return # wait for indicator to start

        self.check_for_entry()
        self.check_for_exit()

    def on_event(self, event: Event):
        if isinstance(event, PositionOpened):
            self.position = self.cache.position(event.position_id)

    def check_for_entry(self):
        # if the current bid is above the hma, we go long
        if self.bid > self.hma.value:
            if self.position and self.position.side == PositionSide.LONG:
                return # already LONG

            order = self.order_factory.market(
                instrument_id = self.instrument_id,
                order_side = OrderSide.BUY,
                quantity = self.trade_size
            )
            self.submit_order(order)
        # if the current bid is below the hma, we go short
        elif self.bid < self.hma.value:
            if self.position and self.position.side == PositionSide.SHORT:
                return # already SHORT

            order = self.order_factory.market(
                instrument_id = self.instrument_id,
                order_side = OrderSide.SELL,
                quantity = self.trade_size
            )
            self.submit_order(order)

    def check_for_exit(self):
        if self.bid >= self.hma.value:
            if self.position and self.position.side == PositionSide.SHORT:
                self.close_position(self.position)
            # If the bid is greater than hma line, close all SHORT positions

        else:
            if self.position and self.position.side == PositionSide.LONG:
                self.close_position(self.position)


    def on_dispose(self):
        pass # do nothing

In [5]:
# Backtest Config
from nautilus_trader.config import BacktestVenueConfig

venue = BacktestVenueConfig(
    name='SIM',
    oms_type='NETTING',
    account_type="MARGIN",
    base_currency='USD',
    starting_balances=['1_000_000 USD']
)

In [6]:
instruments = catalog.instruments(as_nautilus=True)
instruments

[CurrencyPair(id=EUR/USD.SIM, raw_symbol=EUR/USD, asset_class=FX, asset_type=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)]

In [7]:
from nautilus_trader.config import BacktestDataConfig
from nautilus_trader.model.data import QuoteTick

data = BacktestDataConfig(
    catalog_path=(catalog.path),
    data_cls=QuoteTick,
    instrument_id=str(instruments[0].id),
    end_time='2020-01-10',
)

In [8]:
from nautilus_trader.config import BacktestEngineConfig
from nautilus_trader.config import ImportableStrategyConfig

engine = BacktestEngineConfig(
    strategies=[
        ImportableStrategyConfig(
            strategy_path='__main__:HMAStrategy',
            config_path='__main__:HMAConfig',
            config=dict(
                instrument_id=instruments[0].id.value,
                period=16,
                ),
        )
    ],
    logging=LoggingConfig(log_level='ERROR'),
)

In [9]:
from nautilus_trader.config import BacktestRunConfig

config = BacktestRunConfig(
    engine=engine,
    data=[data],
    venues=[venue],
)

In [10]:
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.results import BacktestResult

node = BacktestNode(configs=[config])

results: list[BacktestResult] = node.run()

thread '<unnamed>' panicked at common/src/msgbus.rs:403:48:
called `Result::unwrap()` on an `Err` value: Connection refused (os error 111)
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


In [11]:
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.model.identifiers import Venue

engine: BacktestEngine = node.get_engine(config.id)

engine.trader.generate_order_fills_report()

,trader_id,strategy_id,instrument_id,venue_order_id,position_id,account_id,last_trade_id,type,side,quantity,...,contingency_type,order_list_id,linked_order_ids,parent_order_id,exec_algorithm_id,exec_algorithm_params,exec_spawn_id,tags,ts_init,ts_last
client_order_id,,,,,,,,,,,,,,,,,,,,,
O-20200101-1701-001-000-1,BACKTESTER-001,HMAStrategy-000,EUR/USD.SIM,SIM-1-001,EUR/USD.SIM-HMAStrategy-000,SIM-001,SIM-1-001,MARKET,SELL,1000,...,NO_CONTINGENCY,None,None,None,None,None,None,None,2020-01-01 17:01:21.324000+00:00,2020-01-01 17:01:21.324000+00:00
O-20200101-1717-001-000-2,BACKTESTER-001,HMAStrategy-000,EUR/USD.SIM,SIM-1-002,EUR/USD.SIM-HMAStrategy-000,SIM-001,SIM-1-002,MARKET,BUY,1000,...,NO_CONTINGENCY,None,None,None,None,None,None,None,2020-01-01 17:17:29.270000+00:00,2020-01-01 17:17:29.270000+00:00
O-20200101-1717-001-000-4,BACKTESTER-001,HMAStrategy-000,EUR/USD.SIM,SIM-1-003,EUR/USD.SIM-HMAStrategy-000,SIM-001,SIM-1-003,MARKET,BUY,1000,...,NO_CONTINGENCY,None,None,None,None,None,None,None,2020-01-01 17:17:29.321000+00:00,2020-01-01 17:17:29.321000+00:00
O-20200101-1717-001-000-5,BACKTESTER-001,HMAStrategy-000,EUR/USD.SIM,SIM-1-004,EUR/USD.SIM-HMAStrategy-000,SIM-001,SIM-1-004,MARKET,SELL,1000,...,NO_CONTINGENCY,None,None,None,None,None,None,None,2020-01-01 17:17:29.372000+00:00,2020-01-01 17:17:29.372000+00:00
O-20200101-1717-001-000-7,BACKTESTER-001,HMAStrategy-000,EUR/USD.SIM,SIM-1-005,EUR/USD.SIM-HMAStrategy-000,SIM-001,SIM-1-005,MARKET,SELL,1000,...,NO_CONTINGENCY,None,None,None,None,None,None,None,2020-01-01 17:17:29.473000+00:00,2020-01-01 17:17:29.473000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
O-20200109-2359-001-000-175652,BACKTESTER-001,HMAStrategy-000,EUR/USD.SIM,SIM-1-117102,EUR/USD.SIM-HMAStrategy-000,SIM-001,SIM-1-117102,MARKET,BUY,1000,...,NO_CONTINGENCY,None,None,None,None,None,None,None,2020-01-09 23:59:01.433000+00:00,2020-01-09 23:59:01.433000+00:00
O-20200109-2359-001-000-175654,BACKTESTER-001,HMAStrategy-000,EUR/USD.SIM,SIM-1-117103,EUR/USD.SIM-HMAStrategy-000,SIM-001,SIM-1-117103,MARKET,BUY,1000,...,NO_CONTINGENCY,None,None,None,None,None,None,None,2020-01-09 23:59:41.940000+00:00,2020-01-09 23:59:41.940000+00:00
O-20200109-2359-001-000-175655,BACKTESTER-001,HMAStrategy-000,EUR/USD.SIM,SIM-1-117104,EUR/USD.SIM-HMAStrategy-000,SIM-001,SIM-1-117104,MARKET,SELL,1000,...,NO_CONTINGENCY,None,None,None,None,None,None,None,2020-01-09 23:59:42.322000+00:00,2020-01-09 23:59:42.322000+00:00


In [12]:
engine.trader.generate_positions_report()

,trader_id,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,peak_qty,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl
position_id,,,,,,,,,,,,,,,,,,
EUR/USD.SIM-HMAStrategy-000-23f8d283-d6c5-48b3-bd1f-4aa25fcbfc81,BACKTESTER-001,HMAStrategy-000,EUR/USD.SIM,SIM-001,O-20200101-1701-001-000-1,O-20200101-1717-001-000-2,SELL,FLAT,1000,2020-01-01 17:01:21.324000+00:00,1577899049270000000,2020-01-01 17:17:29.270000+00:00,967946000000,1.12126,1.1216,['0.04 USD'],-0.0003,-0.38 USD
EUR/USD.SIM-HMAStrategy-000-568c09bd-716b-4e9a-9ffa-811a0b79555e,BACKTESTER-001,HMAStrategy-000,EUR/USD.SIM,SIM-001,O-20200101-1717-001-000-4,O-20200101-1717-001-000-5,BUY,FLAT,1000,2020-01-01 17:17:29.321000+00:00,1577899049372000000,2020-01-01 17:17:29.372000+00:00,51000000,1.12166,1.12144,['0.04 USD'],-0.0002,-0.26 USD
EUR/USD.SIM-HMAStrategy-000-f182775e-b0d6-490c-b3ac-2b85124ef697,BACKTESTER-001,HMAStrategy-000,EUR/USD.SIM,SIM-001,O-20200101-1717-001-000-7,O-20200101-1719-001-000-8,SELL,FLAT,1000,2020-01-01 17:17:29.473000+00:00,1577899152771000000,2020-01-01 17:19:12.771000+00:00,103298000000,1.12143,1.12145,['0.04 USD'],-2e-05,-0.06 USD
EUR/USD.SIM-HMAStrategy-000-f5d5d64a-edbd-4619-842f-89a4c5828077,BACKTESTER-001,HMAStrategy-000,EUR/USD.SIM,SIM-001,O-20200101-1719-001-000-10,O-20200101-1719-001-000-11,SELL,FLAT,1000,2020-01-01 17:19:12.822000+00:00,1577899163464000000,2020-01-01 17:19:23.464000+00:00,10642000000,1.12135,1.12151,['0.04 USD'],-0.00014,-0.20 USD
EUR/USD.SIM-HMAStrategy-000-bf39d274-b5c1-46d8-a0e9-04c5988d4314,BACKTESTER-001,HMAStrategy-000,EUR/USD.SIM,SIM-001,O-20200101-1719-001-000-13,O-20200101-1719-001-000-14,BUY,FLAT,1000,2020-01-01 17:19:23.566000+00:00,1577899177961000000,2020-01-01 17:19:37.961000+00:00,14395000000,1.12151,1.12146,['0.04 USD'],-4e-05,-0.09 USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EUR/USD.SIM-HMAStrategy-000-762c0bfc-45e6-4508-9872-ab41a9b26c0c,BACKTESTER-001,HMAStrategy-000,EUR/USD.SIM,SIM-001,O-20200109-2348-001-000-175645,O-20200109-2354-001-000-175646,SELL,FLAT,1000,2020-01-09 23:48:24.146000+00:00,1578614081566000000,2020-01-09 23:54:41.566000+00:00,377420000000,1.1107,1.11075,['0.04 USD'],-5e-05,-0.09 USD
EUR/USD.SIM-HMAStrategy-000-20755b09-acd6-49be-b314-dd8a0dec993d,BACKTESTER-001,HMAStrategy-000,EUR/USD.SIM,SIM-001,O-20200109-2354-001-000-175648,O-20200109-2354-001-000-175649,BUY,FLAT,1000,2020-01-09 23:54:41.678000+00:00,1578614081931000000,2020-01-09 23:54:41.931000+00:00,253000000,1.11078,1.11074,['0.04 USD'],-4e-05,-0.08 USD
EUR/USD.SIM-HMAStrategy-000-f8623f6d-d4e4-4157-893a-d3c535b9a34e,BACKTESTER-001,HMAStrategy-000,EUR/USD.SIM,SIM-001,O-20200109-2354-001-000-175651,O-20200109-2359-001-000-175652,SELL,FLAT,1000,2020-01-09 23:54:42.083000+00:00,1578614341433000000,2020-01-09 23:59:01.433000+00:00,259350000000,1.11074,1.11077,['0.04 USD'],-3e-05,-0.07 USD


In [14]:
engine.trader.generate_account_report(Venue('SIM')) 

,total,locked,free,currency,account_id,account_type,base_currency,margins,reported,info
2020-01-01 17:00:10.447000+00:00,1000000.00,0.00,1000000.00,USD,SIM-001,MARGIN,USD,b'[]',True,b'{}'
2020-01-01 17:01:21.324000+00:00,999999.98,0.00,999999.98,USD,SIM-001,MARGIN,USD,b'[]',False,b'{}'
2020-01-01 17:01:21.324000+00:00,999999.98,33.66,999966.32,USD,SIM-001,MARGIN,USD,"b'[{""type"":""MarginBalance"",""initial"":""0.00"",""m...",False,b'{}'
2020-01-01 17:17:29.270000+00:00,999999.62,33.66,999965.96,USD,SIM-001,MARGIN,USD,"b'[{""type"":""MarginBalance"",""initial"":""0.00"",""m...",False,b'{}'
2020-01-01 17:17:29.270000+00:00,999999.62,0.00,999999.62,USD,SIM-001,MARGIN,USD,b'[]',False,b'{}'
...,...,...,...,...,...,...,...,...,...,...
2020-01-09 23:59:42.322000+00:00,996680.71,0.00,996680.71,USD,SIM-001,MARGIN,USD,b'[]',False,b'{}'
2020-01-09 23:59:54.079000+00:00,996680.69,0.00,996680.69,USD,SIM-001,MARGIN,USD,b'[]',False,b'{}'
2020-01-09 23:59:54.079000+00:00,996680.69,33.34,996647.35,USD,SIM-001,MARGIN,USD,"b'[{""type"":""MarginBalance"",""initial"":""0.00"",""m...",False,b'{}'
2020-01-09 23:59:59.448000+00:00,996680.64,33.34,996647.30,USD,SIM-001,MARGIN,USD,"b'[{""type"":""MarginBalance"",""initial"":""0.00"",""m...",False,b'{}'
